# Data Profiling for all data features

In [57]:
import pandas as pd
import numpy as np 
df = pd.read_csv("https://datasocibmproject.s3.ap-southeast-2.amazonaws.com/structured_data/capital_project_schedules_and_budgets_1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8185 entries, 0 to 8184
Data columns (total 16 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Unnamed: 0                                                  8185 non-null   int64  
 1   project_geographic_district                                 8185 non-null   int64  
 2   project_building_identifier                                 8185 non-null   object 
 3   project_school_name                                         8185 non-null   object 
 4   project_type                                                8185 non-null   object 
 5   project_description                                         8185 non-null   object 
 6   project_phase_name                                          8184 non-null   object 
 7   project_status_name                                         8185 non-null   object 
 8 

In [25]:
df['project_phase_actual_start_date'].head()

0    07/07/2016
1    07/07/2016
2    07/27/2017
3    08/04/2017
4           PNS
Name: project_phase_actual_start_date, dtype: object

In [49]:
len(df['project_phase_actual_end_date'])

8185

# Checking all wrongly formating not in XX/XX/XXXX or PNS or F&E or CM 

In [22]:
def checkDateFormatting(df, col1):
    
    wrong_format_count = 0
    unique_value_wrong_format_dict = {}
    
    # checking format of date for if XX/XX/XXXX or string
    data_type = None

    
    for date in df[col1]:
        correct_format = False
        correct_format, object_type = checkLenghtDataAndInput(date)
        
        if correct_format == False:
            # input directionary 
            unique_value_wrong_format_dict = uniqueWronglyFormatedDatas(unique_value_wrong_format_dict, date)
            wrong_format_count += 1
            
    return wrong_format_count, unique_value_wrong_format_dict

def checkLenghtDataAndInput(data):
    correct_format = False
    # convert string
    data = str(data) 
    object_type = ''
    # XX/XX/XXXX means month/day/year or other acceptable labels 
    if len(data) == 10 and data[2] == '/' and data[5] == '/':
        correct_format = True 
        object_type = 'number'
    elif data == 'PNS' or data == 'F&E' or data == 'CM':
        correct_format = True
        object_type = 'string'
        
        
    return correct_format, object_type

def uniqueWronglyFormatedDatas(dictionary, date):
    if date in dictionary:
        dictionary[date] += 1 
    else:
        dictionary[date] = 1
    return dictionary


In [23]:
# is the date consistent? in the form XX / XX / XXXX

wrong_actual_date_count_start = 0
wrong_actual_unique_dict_start = {}

wrong_actual_date_count_end = 0
wrong_actual_unique_dict_end = {}

wrong_planned_date_count_end = 0
wrong_planned_unique_dict_end = {}


wrong_actual_date_count_start, wrong_actual_unique_dict_start = checkDateFormatting(df, 'project_phase_actual_start_date')
wrong_actual_date_count_end, wrong_actual_unique_dict_end = checkDateFormatting(df,'project_phase_actual_end_date')
wrong_planned_date_count_end, wrong_planned_unique_dict_end = checkDateFormatting(df, 'project_phase_planned_end_date')

print(f"For project_phase_actual_start_date there is total wrongly formated {wrong_actual_date_count_start} and unique wrong values includes {wrong_actual_unique_dict_start} \n")
print(f"For project_phase_actual_end_date there is total wrongly formated {wrong_actual_date_count_end} and unique wrong values includes {wrong_actual_unique_dict_end} \n")
print(f"For project_phase_planned_end_date there is total wrongly formated {wrong_planned_date_count_end} and unique wrong values includes {wrong_planned_unique_dict_end} \n")



For project_phase_actual_start_date there is total wrongly formated 1 and unique wrong values includes {'FTK': 1} 

For project_phase_actual_end_date there is total wrongly formated 2159 and unique wrong values includes {nan: 2158, 'FTK': 1} 

For project_phase_planned_end_date there is total wrongly formated 1324 and unique wrong values includes {'DOES': 318, 'IEH': 652, 'DOEL': 113, 'DOER': 170, 'FTK': 61, 'EMER': 4, 'DOEP': 1, '/  /': 5} 



# Does Correctly Formated dates make logical sense?

In [72]:
class DateRangeChecker:
    def __init__(self):
        pass 

    def dateRangeChecker(self, data):
        # by default
        date_boolean = False 
        
        # Call checks for each column 
        day_boolean = self.daysColumnCheck(data[3:5])
        month_boolean = self.monthColumnsCheck(data[0:2])
        year_boolean = self.yearColumnsCheck(data[6:11])
    
        # Check if all ranges correct
        if day_boolean == True and month_boolean == True and year_boolean == True:
            date_boolean = True 
    
        return date_boolean
        
    def daysColumnCheck(self, data):
        correct_range = False
        days_range = np.arange(0, 32)
        if int(data) in days_range:
            correct_range = True
        return correct_range
    
    def monthColumnsCheck(self, data):
        correct_range = False
        month_range = np.arange(0, 13)
        if int(data) in month_range:
            correct_range = True
        return correct_range
    
    def yearColumnsCheck(self, data):
        correct_range = False
        year_range = np.arange(2000, 2040)
        if int(data) in year_range:
            correct_range = True
        return correct_range



class DataLogicChecker:
    def __init__(self):
        pass
        
    def dataLogicChecker(self, data_col_one_index, data_col_two_index):
        # input : XX/XX/XXXX
        data_logic_boolean = False 
        day_logic_boolean = self.dayLogicChecker(data_col_one_index[3:5], data_col_two_index[3:5])
        month_logic_boolean = self.monthLogicChecker(data_col_one_index[0:2], data_col_two_index[0:2])
        year_logic_boolean = self.yearLogicChecker(data_col_one_index[6:11], data_col_two_index[6:11])
        
        # checks if the logic is correct 
        if (day_logic_boolean and month_logic_boolean and year_logic_boolean ) == True:
            data_logic_boolean = True 

        return data_logic_boolean
        
    def dayLogicChecker(self, day_col_one, day_col_two):
        # Default 
        day_logic_boolean = False 

        # convert to string 'XX' to integer
        day_col_one = int(day_col_one)
        day_col_two = int(day_col_two)
        
        if day_col_one < day_col_two:
            day_logic_boolean = True
            
        return day_logic_boolean

    def monthLogicChecker(self, month_col_one, month_col_two):
        month_logic_boolean = False 

        # Convert to integer 
        month_col_one = int(month_col_one)
        month_col_two = int(month_col_two)

        # Check if logic is true
        if month_col_one < month_col_two:
            month_logic_boolean = True 
            
        return month_logic_boolean 

    def yearLogicChecker(self, year_col_one, year_col_two):
        year_logic_boolean = False

        # Convery to year XXXX to integer 
        year_col_one = int(year_col_one)
        year_col_two = int(year_col_two)
    
        # Check if logic is true
        if year_col_one < year_col_two:
            year_logic_boolean = True
            
        return year_logic_boolean 


# Note this function has order so col1 has to be before col2 in a date sense 
# This function checks if correct input date, then checks range of date is correct then checks logic
def dateLogicSence(df, col1, col2):
    # check ranges of day, month and year, hen check logic of both cols1 and cols2 
    logic_checker = False
    dataRangeChecker = DateRangeChecker()
    dataLogicChecker = DataLogicChecker()

    # iterates over all index in both col1 and col2 to check ranage and logic 
    for index in range(len(df)):
        count_col_one, object_type_col_one = checkLenghtDataAndInput(df[col1].iloc[index])
        count_col_two, object_type_col_two = checkLenghtDataAndInput(df[col2].iloc[index])

        # given both columns are in form XX/XX/XXXX 
        if count_col_one and count_col_two and object_type_col_one == 'number' and object_type_col_two == 'number':
            col1_data_range_boolean = dataRangeChecker.dateRangeChecker(df[col1].iloc[index])
            col2_data_range_boolean = dataRangeChecker.dateRangeChecker(df[col2].iloc[index])

            # after checking date is whichin range
            if col1_data_range_boolean == True and col2_data_range_boolean == True:
                logic_checker = dataLogicChecker.dataLogicChecker(df[col1].iloc[index], df[col2].iloc[index])
                
    
        return logic_checker
        
    return logic_checker
    
# Calling above function and classes
logic_checker_actual_start_end_dates = dateLogicSence(df,'project_phase_actual_start_date' , 'project_phase_actual_end_date')
logic_checker_actual_start_actual_end_dates = dateLogicSence(df,'project_phase_actual_start_date' , 'project_phase_planned_end_date')

print(logic_checker_actual_start_end_dates)
print(logic_checker_actual_start_actual_end_dates)


False
False
